# Segmenting and Clustering Neighborhoods in Toronto

# Instructions
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

# Part 1

The part 1 consists in scraping the data from a wiki url and pre processing the information recovered

In [1]:
import numpy as np
import pandas as pd

URL read with pandas and encoding 'Not assigned' values as NaN

In [2]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, na_values= 'Not assigned')[0]

Filtering out the rows with NaN values on the column 'Borough':

In [3]:
df = df[df['Borough'].notna()]

Restarting the dataframe index:

In [4]:
df.reset_index(inplace=True, drop=True)

Grouping the data using the postal code:

In [5]:
df_grouped = df.groupby(by='Postal Code').first()
df_grouped.reset_index(inplace=True)
df_grouped.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Replacing the 'Not assigned' value on Neighbourhood by the respective 'Borough':

In [6]:
df_grouped['Neighbourhood'].replace('Not assigned', df_grouped['Borough'])
df_grouped.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_grouped.shape

(103, 3)

# Part 2

Load new geolocation data of Toronto, merge with the dataframe created in the Part 1 in order to create a unified dataframe

Import the geocoder lib and folium lib:

In [55]:
import geocoder # geographical data

Definition of the geodata search function:

In [9]:
def geodataSearch(postal_code):
    from numpy import nan
    lat_lng_coords = None
    num_calls = 0
    LIMIT = 10
    while(lat_lng_coords is None and num_calls < LIMIT):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        num_calls += 1
    
    if(num_calls >= LIMIT):
        print('Google is not responding')
        return [nan, nan]

    return lat_lng_coords

In [10]:
latitude, longitude = geodataSearch('M5G')

Google is not responding


Since geocoder is no longer responding to calls, I'll work with the csv data indicated in the course website:

In [11]:
geodata = pd.read_csv('Geospatial_Coordinates.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Creating a unified dataframe from informations in df_grouped and geodata

In [12]:
df = pd.concat([df_grouped,geodata[['Latitude','Longitude']]], axis=1)
df.head(15)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Part 3

Filtering the dataframe to only the Boroughs that contain the word 'Toronto' on it's name:

In [29]:
df_filtered =df[df['Borough'].str.contains('Toronto')]
df_filtered.reset_index(inplace=True, drop=True)
df_filtered.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Verifying the dataframe size:

In [35]:
print('The dataframe has {} boroughs and {} neighbourhoods'.format(len(df_filtered['Borough'].unique()),
df_filtered.shape[0]))

The dataframe fas 4 boroughs and 39 neighbourhoods


Use geopy lib to get the latitude and logitude values of Toronto, and folium to render toronto map:

In [56]:
from geopy.geocoders import Nominatim # geographical data
import folium  # map rendering lib

In [132]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoos superimposed on top

In [133]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to map

for lat, lng, borough, neighborhood in zip(df_filtered['Latitude'],df_filtered['Longitude'], df_filtered['Borough'], df_filtered['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)

map_toronto

![Screenshot from folium 1](./SC_toronto_map1.png)

Exploring Toronto's neighbourhoods using Foursquare geo API:

In [101]:
import json # lib to handle json files
import requests # lib to handle requests
from pandas.io.json import json_normalize # convert JSON file into a pandas dataframe

# File created in order to protect my access credentials
from foursquare_credentials import CLIENT_ID, CLIENT_SECRET

VERSION = '20180605' # API version
LIMIT = 100 # Default API limite value
RADIUS = 500
df_filtered['Neighbourhood'].unique()

array(['The Beaches', 'The Danforth West, Riverdale',
       'India Bazaar, The Beaches West', 'Studio District',
       'Lawrence Park', 'Davisville North',
       'North Toronto West, Lawrence Park', 'Davisville',
       'Moore Park, Summerhill East',
       'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park',
       'Rosedale', 'St. James Town, Cabbagetown', 'Church and Wellesley',
       'Regent Park, Harbourfront', 'Garden District, Ryerson',
       'St. James Town', 'Berczy Park', 'Central Bay Street',
       'Richmond, Adelaide, King',
       'Harbourfront East, Union Station, Toronto Islands',
       'Toronto Dominion Centre, Design Exchange',
       'Commerce Court, Victoria Hotel', 'Roselawn',
       'Forest Hill North & West, Forest Hill Road Park',
       'The Annex, North Midtown, Yorkville',
       'University of Toronto, Harbord',
       'Kensington Market, Chinatown, Grange Park',
       'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bath

## **Let's explore the 'Central Bay Street' neighbourhood:**

In [92]:
neighborhood_name = 'Central Bay Street'
neighborhood_index = df_filtered.index[df_filtered['Neighbourhood'] == neighborhood_name].tolist()[0]
neighborhood_latitude = df_filtered.loc[neighborhood_index,'Latitude']
neighborhood_longitude = df_filtered.loc[neighborhood_index,'Longitude']

print('Latitude and longitude values of {} are {},{}.'.format(
    neighborhood_name, 
    neighborhood_latitude, 
    neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524,-79.3873826.


Creating the get request URL for exploring the venues in the neighbourhood:

In [97]:
url_base = 'https://api.foursquare.com/v2/'
LIMIT = 100
RADIUS = 500

url = url_base + 'venues/explore?client_id={}&client_secret={}&v=20170511&ll={},{}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET,neighborhood_latitude,neighborhood_longitude,LIMIT,RADIUS)

results = requests.get(url).json()

Function from Foursquare lab to extract the categories from the results

In [99]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [112]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns names
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

# clean columns
nearby_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870


In [113]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

68 venues were returned by Foursquare.


In [130]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14.5)

# add markers to map

for lat, lng, name, categorie in zip(nearby_venues['lat'],nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categorie)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)

map_toronto

![Screenshot from folium 2](./SC_toronto_map2.png)

## **Now let's explore and cluster Toronto neighbourhoods:**

Reusing the function given in a previus notebook. The function porpouse is process all the neighbourhoods data recovered using the Foursquare API:

In [131]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Running the function for each neighbouhood in the filtered dataframe (only boroughs whose names contains 'Toronto') and creating a new dataframe:

In [135]:
toronto_venues = getNearbyVenues(df_filtered['Neighbourhood'], df_filtered['Latitude'], df_filtered['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

Checking the size and first lines of the resulting dataframe:

In [137]:
print(toronto_venues.shape)
toronto_venues.head()

(1624, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues for each neighbourhood:

In [143]:
toronto_venues.groupby(by='Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


How many unique categories in the dataframe?

In [145]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


### Analyzing each Neighbourhood

In [187]:
# Applying one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

# add neighbourdhood back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to first column
#fixed_columns = list(toronto_onehot.columns)
#toronto_onehot = toronto_onehot[fixed_columns]
fixed_columns = ['Neighborhood'] +[col for col in toronto_onehot.columns if col != 'Neighborhood' ]
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Checking the new dataframe size:

In [188]:
toronto_onehot.shape

(1624, 237)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [189]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.018182,0.0,0.000000,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.014706,0.0,0.000000,0.014706,0.0,0.014706
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
6,Church and Wellesley,0.013333,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013333,0.0,...,0.013333,0.000000,0.0,0.0,0.000000,0.0,0.013333,0.000000,0.0,0.026667
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.030000,0.0,...,0.000000,0.000000,0.0,0.0,0.020000,0.0,0.000000,0.010000,0.0,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.030303,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000


Let's confirm the new size:

In [191]:
toronto_grouped.shape

(39, 237)

Reusing the function to sort venues in descending order

In [192]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a new dataframe with the top 10 venues for each neighborhood

In [193]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Bistro,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Pizza Place,Garden Center,Farmers Market,Butcher,Brewery,Burrito Place,Recording Studio,Garden,Auto Workshop,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Plane,Airport,Airport Food Court,Airport Gate,Airport Terminal,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Department Store,Burger Joint,Japanese Restaurant,Thai Restaurant


### Clustering Neighborhoods
Let's run the k-mean to cluster the neighborhood into 5 cluster

In [194]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [198]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-mens clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0, 1, 0,
       0, 0, 0, 0, 3, 4, 0, 0], dtype=int32)

Creating a new dataframe including the cluster and the top 10 venues for each neighborhood

In [203]:
# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

toronto_merged = df_filtered

# merge toronto_grouped with df_filtered to add latitude, longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Ice Cream Shop,Furniture / Home Store,Pub,Spa,Indian Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Gym,Pub,Sandwich Place,Liquor Store,Burrito Place,Italian Restaurant,Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Bookstore,Seafood Restaurant,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Swim School,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Let's visualize the resulting clusters

In [208]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [216]:
# create map
map_cluster = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)

map_cluster

![Screenshot from folium 3](./SC_toronto_map3.png)

### Examine Clusters

Let's take a look on each cluster

**Cluster 1**

In [217]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Pub,Trail,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Ice Cream Shop,Furniture / Home Store,Pub,Spa,Indian Restaurant
2,East Toronto,0,Park,Gym,Pub,Sandwich Place,Liquor Store,Burrito Place,Italian Restaurant,Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,East Toronto,0,Coffee Shop,Bakery,Gastropub,Brewery,Café,American Restaurant,Convenience Store,Bookstore,Seafood Restaurant,Cheese Shop
5,Central Toronto,0,Department Store,Dog Run,Park,Gym / Fitness Center,Breakfast Spot,Dance Studio,Hotel,Sandwich Place,Food & Drink Shop,Discount Store
6,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Salon / Barbershop,Fast Food Restaurant,Spa,Diner,Sporting Goods Shop,Restaurant
7,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Coffee Shop,Café,Sushi Restaurant,Pharmacy,Seafood Restaurant
9,Central Toronto,0,Coffee Shop,Pizza Place,Light Rail Station,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Supermarket,Fried Chicken Joint
11,Downtown Toronto,0,Coffee Shop,Café,Pizza Place,Restaurant,Italian Restaurant,Market,Park,Chinese Restaurant,Bakery,Pub
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio


**Cluster 2**

In [218]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Playground,Trail,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


**Cluster 3**

In [219]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Park,Bus Line,Swim School,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


**Cluster 4**

In [220]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Park,Playground,Trail,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


**Cluster 5**

In [221]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, 
toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
